In [ ]:
record_button = widgets.Button(description='New Session', disabled=True, button_style='', icon='circle')
stop_button = widgets.Button(description='Stop', disabled=True, button_style='', icon='square')

session_id = widgets.Text(value='None', placeholder='', description='Session ID:', disabled=True)
record_status = widgets.Text(value='None', placeholder='', description='Recording:', disabled=True)

async def f():
    last_heard_time = 0

    while True:
        latest_msgs = {}

        try:
            # for i in range(1000):
            #     topic, data = ZMQAMH.sub_socket.recv_multipart(flags=zmq.NOBLOCK)
            #     latest_msgs[topic] = data
            topic, data = await ZMQAMH.sub_socket.recv_multipart()
            latest_msgs[topic] = data
        except zmq.ZMQError:
            pass

        for topic, data in latest_msgs.items():
            if topic == b"/loggerd/status/":
                last_heard_time = time.time()
                record_button.disabled = False
                stop_button.disabled = False
                with messages.LoggerStatus.from_bytes(data) as msg:
                    session_id.value = msg.sessionId if msg.recording else "no session"
                    record_status.value = str(msg.recording)
                    record_button.button_style = 'danger' if msg.recording else ''

        if time.time() - last_heard_time > 3:
            session_id.value = "Disconnected"
            record_status.value = "Disconnected"
            record_button.disabled = True
            stop_button.disabled = True

        await asyncio.sleep(0.1)

try:
    ui_update_task.cancel()
except:
    pass
ui_update_task = asyncio.create_task(f())

async def on_record_button_clicked(b):
    await ZMQAMH.pub_socket.send_multipart([b"/loggerd/start/", b""])
    print(f"Record button clicked, b = {b}")
    
async def on_stop_button_clicked(b):
    await ZMQAMH.pub_socket.send_multipart([b"/loggerd/stop/", b""])
    print(f"Stop button clicked, b = {b}")

colortoggle = widgets.ToggleButtons(
    options=[('Opaque', 0), ('Transparent' ,1)],
    description='Patient Material:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

viewindicator = widgets.ToggleButtons(
    options=[('On',True), ('Off',False)],
    description='View Indicator ON/OFF:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

selfviewindicator = widgets.ToggleButtons(
    options=[('On',True), ('Off',False)],
    description='View Indicator Self-Select ON/OFF:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

flippatient = widgets.ToggleButtons(
    options=[('Original',True), ('Flipped',False)],
    description='Flip Patient:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

kwireindicator = widgets.ToggleButtons(
    options=[('On',True), ('Off',False)],
    description='Kwire Error Indicator ON/OFF:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

kwireerrorselection = widgets.ToggleButtons(
    options=[('On',True), ('Off',False)],
    description='Kwire Error Indicator Self-Selection',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)


corridorselection= widgets.Select(
    options=[('Left S1', 0), ('Left Ramus', 1), ('L Ramus Short', 2),('L Teardrop', 3),('Right S1', 4), ('Right Ramus', 5), ('Right Ramus Short', 6),('Right Teardrop', 7),('S 2', 8),('S 3', 9),('None',100)],
    value=2,
    description='Corridor:',
)


update_setting = widgets.Button(
    description='Update Setting',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

viewindicatorselect = widgets.SelectMultiple(
    options=['Anteroposterior_View', 'Lateral_View', 'Inlet_View', 'Outlet_View', 'Obturator_Oblique_Left/Iliac_Oblique_Right', 'Obturator_Oblique_Right/Iliac_Oblique_Left', 'Obturator_Oblique_Inlet_Left/Iliac_Oblique_Inlet_Right','Obturator_Oblique_Outlet_Left/Iliac_Oblique_Outlet_Right','Obturator_Oblique_Inlet_Right/Iliac_Oblique_Inlet_Left','Obturator_Oblique_Outlet_Right/Iliac_Oblique_Outlet_Left', 'Teardrop_Left_View','Teardrop_Right_View','None'],
    value=['Lateral_View'],
    #rows=10,
    description='View Indicator Selection:',
    disabled=False
)

patient_model_layout = widgets.VBox([colortoggle,flippatient])
view_indicator_layout = widgets.VBox([viewindicator,selfviewindicator,viewindicatorselect])
surgical_instrument_layout = widgets.VBox([kwireindicator,kwireerrorselection,corridorselection])
display(widgets.HBox([patient_model_layout,view_indicator_layout ,surgical_instrument_layout,update_setting]))

async def updatesetting_clicked(b):
    response_topic = "/mp/setting/webgui/SettingManager/"
    msg = messages.SycnedSetting.new_message()
    msg.timestamp = msg.timestamp = float(np.float64(time.time()).item())
    msg.clientId = 'webgui'
    uiControlVar = msg.setting.init("uiControl")
    uiControlVar.patientMaterial = colortoggle.value
    uiControlVar.flippatient = flippatient.value #need add      
    uiControlVar.carmIndicator = viewindicator.value
    uiControlVar.viewIndicatorselfselect = selfviewindicator.value #need add
    uiControlVar.annotationError = viewindicatorselect.value#for different view selection
    uiControlVar.corridorIndicator =kwireindicator.value
    uiControlVar.webcorridorerrorselect = kwireerrorselection.value
    uiControlVar.webcorridorselection = corridorselection.value
    await ZMQAMH.pub_socket.send_multipart([response_topic.encode(), msg.to_bytes()])
    print(f"Update Setting button clicked, b = {b}")

update_setting.on_click(lambda b: asyncio.create_task(updatesetting_clicked(b)))
record_button.on_click(lambda b: asyncio.create_task(on_record_button_clicked(b)))
record_button.on_click(lambda b: asyncio.create_task(updatesetting_clicked(b)))
stop_button.on_click(lambda b: asyncio.create_task(on_stop_button_clicked(b)))



display(widgets.HBox([record_button, stop_button, session_id, record_status]))